In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [3]:
def scale(df):
    scaler = StandardScaler()
    scaler.fit(df)
    #print(scaler.mean_)
    return scaler.transform(df)

In [4]:
def encode_values(df):
    le = preprocessing.LabelEncoder()
    le.fit(df)
    print(len(le.classes_))
    return le.transform(df)

In [5]:
def preprocess(dataset):
    for col in ['Geography']:
        dataset[col] = encode_values(dataset[col].values)
    dataset = dataset.drop(['CustomerId'],axis=1)
    dataset = dataset.drop(['Surname'],axis=1)
    dataset = dataset.drop(['Gender'],axis=1)
    if 'Exited' in dataset.columns:
        temp_dataset = scale(dataset.drop('Exited',axis=1).values)
        new_dataset = pd.DataFrame(temp_dataset,columns=['CreditScore', 'Geography', 'Age', 'Tenure', 'Balance',
               'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'])
        new_dataset['Exited'] = dataset['Exited']
    else:
        temp_dataset = scale(dataset.values)
        new_dataset = pd.DataFrame(temp_dataset,columns=['CreditScore', 'Geography', 'Age', 'Tenure', 'Balance',
               'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'])
    sample = new_dataset.sample(frac=1)
    return sample

In [6]:
dataset = pd.read_csv('dataset.csv')
train_dataset = preprocess(dataset)
Y = train_dataset.Exited.values
X = train_dataset.drop('Exited',axis=1).values
X = X.reshape(X.shape[0],X.shape[1],1)
Y = Y.reshape(Y.shape[0],1)

3


In [12]:
np.random.seed(1234)
np.seterr(over='ignore')

class ArtificialNeuralNetwork():
    def __init__(self):
        np.random.seed(1)
        self.weights = {}
        self.num_layers = 1
        self.adjustments = {}

    def add_layer(self, shape):
        self.weights[self.num_layers] = np.vstack((2 * np.random.random(shape) - 1, 2 * np.random.random((1, shape[1])) - 1))
        self.adjustments[self.num_layers] = np.zeros(shape)
        self.num_layers += 1

    def __sigmoid(self, x):
        x = np.clip(x,-20,20)
        return 1 / (1 + np.exp(-x))

    def __sigmoid_derivative(self, x):
        return x * (1 - x)

    def predict(self, data):
        for layer in range(2, self.num_layers+1):
            data = np.dot(data.T, self.weights[layer-1][:-1, :]) + self.weights[layer-1][ -1,:].T
            data = self.__sigmoid(data).T
        return data

    def forward_propagate(self, data):
        activation_values = {}
        activation_values[1] = data
        for layer in range(2, self.num_layers+1):
            data = np.dot(data.T, self.weights[layer-1][:-1, :]) + self.weights[layer-1][-1, :].T # + self.biases[layer]
            data = self.__sigmoid(data).T
            activation_values[layer] = data
        return activation_values

    def simple_error(self, outputs, targets):
        return outputs-targets

    def sum_squared_error(self, outputs, targets):
        return 0.5 * np.mean(np.sum(np.power(outputs - targets, 2), axis=1))
    
    def binaryentropy(self,outputs,targets):
        return -(1.0/len(targets)) * np.sum(targets*np.log(outputs) + (1-targets)*np.log(1-outputs))

    def __back_propagate(self, output, target):
        deltas = {}
        deltas[self.num_layers] = self.simple_error(output[self.num_layers],target)

        # Delta of hidden Layers
        for layer in reversed(range(2, self.num_layers)):  # All layers except input/output
            a_val = output[layer]
            weights = self.weights[layer][:-1, :]
            prev_deltas = deltas[layer+1]
            deltas[layer] = np.multiply(np.dot(weights, prev_deltas), self.__sigmoid_derivative(a_val))

        # Caclculate total adjustements based on deltas
        for layer in range(1, self.num_layers):
            self.adjustments[layer] += np.dot(deltas[layer+1], output[layer].T).T

    def __gradient_descente(self, batch_size, learning_rate):
        # Calculate partial derivative and take a step in that direction
        for layer in range(1, self.num_layers):
            partial_d = (1/batch_size) * self.adjustments[layer]
            self.weights[layer][:-1, :] += learning_rate * -partial_d
            self.weights[layer][-1, :] += learning_rate *1e-3* -partial_d[-1, :]

    def train(self, inputs, targets, num_epochs, learning_rate=0.01, stop_accuracy=1e-3):
        error = []
        for iteration in range(num_epochs):
            for i in range(len(inputs)):
                x = inputs[i]
                y = targets[i]
                # Pass the training set through our neural network
                output = self.forward_propagate(x)

                # Calculate the error
                loss = self.sum_squared_error(output[self.num_layers], y)
                error.append(loss)

                # Calculate Adjustements
                self.__back_propagate(output, y)

            self.__gradient_descente(i, learning_rate)

            # Check if accuarcy criterion is satisfied
            if iteration > 0 and iteration %1000 == 0:
                print( iteration,np.mean(np.abs(error[-(i+1):])))
            if np.mean(np.abs(error[-(i+1):])) < stop_accuracy and iteration > 0:
                print( iteration,np.mean(np.abs(error[-(i+1):])))

        return(np.asarray(error), iteration+1)

In [13]:
# Create instance of a neural network
nn = ArtificialNeuralNetwork()
nn.add_layer((9, 7))
nn.add_layer((7, 5))
nn.add_layer((5, 1))

In [14]:
#Training Error
training_data = X
training_labels = Y
print(training_labels.shape,training_data.shape)
error, iteration = nn.train(training_data, training_labels,10000,learning_rate=0.005)
print('Error = ', np.mean(np.abs(error[-4:])))
print('Epoches needed to train = ', iteration)

(9000, 1) (9000, 9, 1)
1000 0.08091833634075879
2000 0.07395474045337237
3000 0.06876141675726224
4000 0.0691624085085026
5000 0.06830072362992386
6000 0.06921415646942225
7000 0.06845587213263242
8000 0.06887618720339578
9000 0.0682674737526812
Error =  0.08896786203155717
Epoches needed to train =  10000


In [15]:
test_dataset = pd.read_csv("judge.csv")
test_dataset = preprocess(test_dataset)
X = test_dataset.values
X = X.reshape(1000,9,1)

3


In [16]:
testing_data = X
output_testdata = []
for i in range(len(testing_data)):
    res = nn.predict(testing_data[i])[0]
    if res[0] >= 0.5:
        output_testdata.append(1)
    elif res[0] < 0.5:
        output_testdata.append(0)
data = pd.read_csv('judge.csv')
Result_dataframe = pd.DataFrame()
Result_dataframe['CustomerId'] = data['CustomerId']
Result_dataframe['Exited'] = output_testdata
Result_dataframe.to_csv("judge-pred.csv",index=False)